In [28]:
#Working pieces. Need to add Diagonal and nakedTwins

rows = 'ABCDEFGHI'
cols = '123456789'

assignments = []

def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:the values dictionary with the naked twins eliminated from peers.
    """
    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers
    
    pass

######################################################################
##UTILS
######################################################################

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s+t for s in A for t in B]

boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args: grid(string) - A grid in string form.
    Returns: A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value,
            then the value will be '123456789'.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def eliminate(values):
    """Eliminate values from peers of each box with a single value.
    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.
    Args:values: Sudoku in dictionary form.
    Returns:Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values=[box for box in values.keys() if len(values[box])==1]
    for box in solved_values:
        digit=values[box]
        for peer in peers[box]:
            values[peer]=values[peer].replace(digit,'')
    return values

def only_choice(values):
    '''Finalize all values that are the only choice for a unit.
    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.
    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    '''
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

def reduce_puzzle(values):
    '''Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    '''
    stalled=False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

def search(values):
    "Using depth-first search and propagation, try all possible values."
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes):
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

def solve(grid):
    """Find the solution to a Sudoku grid.
    Args: grid(string): a string representing a sudoku grid.
        Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns: The dictionary representation of the final sudoku grid.
            False if no solution exists.
    """
    values=grid_values(grid)
    '''
    print("Given sudoku:")
    display(values)
    values=eliminate(values)
    print("=====================")
    print("After eliminate:")
    display(values)
    print("=====================")
    print("After onlychoice:")
    values= only_choice(values)
    display(values)
    '''
    #print("=====================")
    #print("After reduce:")
    values=reduce_puzzle(values)
    #display(values)

    values=search(values)
    #print("=====================")
    #print("After search:")
    #display(values)
    
    return values

if __name__ == '__main__':
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    reducegrid= '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
    searchneededgrid = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
    
    print("After solve:")
    display(solve(diag_sudoku_grid))


    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


After solve:
2 3 9 |8 7 4 |1 5 6 
7 5 4 |3 1 6 |2 9 8 
6 8 1 |9 5 2 |3 7 4 
------+------+------
4 7 6 |1 2 8 |5 3 9 
3 1 2 |4 9 5 |6 8 7 
5 9 8 |6 3 7 |4 1 2 
------+------+------
1 4 3 |7 6 9 |8 2 5 
9 6 5 |2 8 3 |7 4 1 
8 2 7 |5 4 1 |9 6 3 
We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.


In [42]:
#Working pieces. Need to add Diagonal and nakedTwins
#refered to https://github.com/vxy10/AIND-Sudoku/blob/master/solution.py

rows = 'ABCDEFGHI'
cols = '123456789'

assignments = []

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s+t for s in A for t in B]

boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]


#Adding diagonal sudoku
diagonal1_units = [[rows[i]+cols[i] for i in range(len(rows))]]
diagonal2_units = [[rows[i]+cols[::-1][i] for i in range(len(rows))]]

isDiagonal = 1 #0 for non-diagonal sudoku
if isDiagonal == 1:
    unitlist = row_units + column_units + square_units + diagonal1_units + diagonal2_units
else:
    unitlist = row_units + column_units + square_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)
    
def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args: grid(string) - A grid in string form.
    Returns: A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value,
            then the value will be '123456789'.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def eliminate(values):
    """Eliminate values from peers of each box with a single value.
    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.
    Args:values: Sudoku in dictionary form.
    Returns:Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values=[box for box in values.keys() if len(values[box])==1]
    for box in solved_values:
        digit=values[box]
        for peer in peers[box]:
            values[peer]=values[peer].replace(digit,'')
    return values

def only_choice(values):
    '''Finalize all values that are the only choice for a unit.
    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.
    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    '''
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

def reduce_puzzle(values):
    '''Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    '''
    stalled=False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

def search(values):
    "Using depth-first search and propagation, try all possible values."
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes):
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt
def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:the values dictionary with the naked twins eliminated from peers.
    """
    # Find all instances of naked twins
    all_twins = [box for box in values.keys() if len(values[box]) == 2]
    naked_twins = [[box1,box2] for box1 in all_twins \
                    for box2 in peers[box1] \
                    if set(values[box1])==set(values[box2]) ]
    #print(naked_twins)
    
    # Eliminate the naked twins as possibilities for their peers
    for i in range(len(naked_twins)):
        box1 = naked_twins[i][0]
        box2 = naked_twins[i][1]
        # Intersection of peers
        peers1 = set(peers[box1])
        peers2 = set(peers[box2])
        peers_int = peers1 & peers2
        # Delete the two digits in naked twins from all common peers.
        for peer_val in peers_int:
            if len(values[peer_val])>2:
                for rm_val in values[box1]:
                    values = assign_value(values, peer_val, values[peer_val].replace(rm_val,''))
    return values

def solve(grid):
    """Find the solution to a Sudoku grid.
    Args: grid(string): a string representing a sudoku grid.
        Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns: The dictionary representation of the final sudoku grid.
            False if no solution exists.
    """
    values=grid_values(grid)
    '''
    print("Given sudoku:")
    display(values)
    
    values=eliminate(values)
    print("=====================")
    print("After eliminate:")
    display(values)
    print("=====================")
    print("After onlychoice:")
    values= only_choice(values)
    display(values)
    print("After nakedtwins:")
    values= naked_twins(values)
    display(values)
    '''
    ''''''
    #print("=====================")
    #print("After reduce:")
    values=reduce_puzzle(values) # eliminate, only_choice, naked_twins
    #display(values)
    
    values=search(values)
    #print("=====================")
    #print("After search:")
    #display(values)
    
    return values

if __name__ == '__main__':
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    reducegrid= '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
    searchneededgrid = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
    
    print("After solve:")
    display(solve(diag_sudoku_grid))


    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


After solve:
2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 
We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.


In [49]:
import json
import pprint
pp = pprint.PrettyPrinter(indent=2)

with open('sudoku-result-16829.json') as json_data:
    d = json.load(json_data)
    pp.pprint(d)

{ 'tests': [ { 'advice': 'Try the following example and see what happens:\n'
                         '{"G7": "2345678", "G6": "1236789", "G5": "23456789", '
                         '"G4": "345678", "G3": "1234569", "G2": "12345678", '
                         '"G1": "23456789", "G9": "24578", "G8": "345678", '
                         '"C9": "124578", "C8": "3456789", "C3": "1234569", '
                         '"C2": "1234568", "C1": "2345689", "C7": "2345678", '
                         '"C6": "236789", "C5": "23456789", "C4": "345678", '
                         '"E5": "678", "E4": "2", "F1": "1", "F2": "24", "F3": '
                         '"24", "F4": "9", "F5": "37", "F6": "37", "F7": "58", '
                         '"F8": "58", "F9": "6", "B4": "345678", "B5": '
                         '"23456789", "B6": "236789", "B7": "2345678", "B1": '
                         '"2345689", "B2": "1234568", "B3": "1234569", "B8": '
                         '"3456789", "B9": "124578", "I9":

In [97]:
#Working pieces. Need to add Diagonal and nakedTwins
#refered to https://github.com/vxy10/AIND-Sudoku/blob/master/solution.py

rows = 'ABCDEFGHI'
cols = '123456789'

assignments = []

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s+t for s in A for t in B]

boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]


#Adding diagonal sudoku
diagonal1_units = [[rows[i]+cols[i] for i in range(len(rows))]]
diagonal2_units = [[rows[i]+cols[::-1][i] for i in range(len(rows))]]

isDiagonal = 1 #0 for non-diagonal sudoku
if isDiagonal == 1:
    unitlist = row_units + column_units + square_units + diagonal1_units + diagonal2_units
else:
    unitlist = row_units + column_units + square_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)
    
def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args: grid(string) - A grid in string form.
    Returns: A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value,
            then the value will be '123456789'.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def eliminate(values):
    """Eliminate values from peers of each box with a single value.
    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.
    Args:values: Sudoku in dictionary form.
    Returns:Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values=[box for box in values.keys() if len(values[box])==1]
    for box in solved_values:
        digit=values[box]
        for peer in peers[box]:
            values[peer]=values[peer].replace(digit,'')
    return values

def only_choice(values):
    '''Finalize all values that are the only choice for a unit.
    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.
    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    '''
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

def reduce_puzzle(values):
    '''Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    '''
    stalled=False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

def search(values):
    "Using depth-first search and propagation, try all possible values."
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes):
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt
def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:the values dictionary with the naked twins eliminated from peers.
    """
    # Find all instances of naked twins
    all_twins = [box for box in values.keys() if len(values[box]) == 2]
    naked_twins = [[box1,box2] for box1 in all_twins \
                    for box2 in peers[box1] \
                    if set(values[box1])==set(values[box2]) ]
    print(naked_twins)
    
    # Eliminate the naked twins as possibilities for their peers
    for i in range(len(naked_twins)):
        box1 = naked_twins[i][0]
        box2 = naked_twins[i][1]
        # Intersection of peers
        peers1 = set(peers[box1])
        peers2 = set(peers[box2])
        peers_int = peers1 & peers2
        # Delete the two digits in naked twins from all common peers.
        for peer_val in peers_int:
            if len(values[peer_val])>2:
                for rm_val in values[box1]:
                    values = assign_value(values, peer_val, values[peer_val].replace(rm_val,''))
    return values

def solve(grid):
    """Find the solution to a Sudoku grid.
    Args: grid(string): a string representing a sudoku grid.
        Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns: The dictionary representation of the final sudoku grid.
            False if no solution exists.
    """
    values=grid_values(grid)
    print(values)
    
    #print("Given sudoku:")
    #display(values)
    '''
    values=eliminate(values)
    print("After eliminate:")
    display(values)
    print("After onlychoice:")
    values= only_choice(values)
    display(values)
    print("After nakedtwins:")
    values= naked_twins(values)
    display(values)
    '''
    ''''''
    #print("After reduce:")
    values=reduce_puzzle(values) # eliminate, only_choice, naked_twins
    #display(values)
    
    values=search(values)
    #print("After search:")
    #display(values)
    
    return values

if __name__ == '__main__':
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    reducegrid= '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
    searchneededgrid = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
    
    print("After solve:")
    display(solve(diag_sudoku_grid))


    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


After solve:
{'A1': '2', 'A2': '123456789', 'A3': '123456789', 'A4': '123456789', 'A5': '123456789', 'A6': '123456789', 'A7': '123456789', 'A8': '123456789', 'A9': '123456789', 'B1': '123456789', 'B2': '123456789', 'B3': '123456789', 'B4': '123456789', 'B5': '123456789', 'B6': '6', 'B7': '2', 'B8': '123456789', 'B9': '123456789', 'C1': '123456789', 'C2': '123456789', 'C3': '1', 'C4': '123456789', 'C5': '123456789', 'C6': '123456789', 'C7': '123456789', 'C8': '7', 'C9': '123456789', 'D1': '123456789', 'D2': '123456789', 'D3': '6', 'D4': '123456789', 'D5': '123456789', 'D6': '8', 'D7': '123456789', 'D8': '123456789', 'D9': '123456789', 'E1': '3', 'E2': '123456789', 'E3': '123456789', 'E4': '123456789', 'E5': '9', 'E6': '123456789', 'E7': '123456789', 'E8': '123456789', 'E9': '7', 'F1': '123456789', 'F2': '123456789', 'F3': '123456789', 'F4': '6', 'F5': '123456789', 'F6': '123456789', 'F7': '4', 'F8': '123456789', 'F9': '123456789', 'G1': '123456789', 'G2': '4', 'G3': '123456789', 'G4': '

In [151]:
#Working pieces. Need to add Diagonal and nakedTwins
#refered to https://github.com/vxy10/AIND-Sudoku/blob/master/solution.py

rows = 'ABCDEFGHI'
cols = '123456789'

assignments = []

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s+t for s in A for t in B]

boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]


#Adding diagonal sudoku
diagonal1_units = [[rows[i]+cols[i] for i in range(len(rows))]]
diagonal2_units = [[rows[i]+cols[::-1][i] for i in range(len(rows))]]

isDiagonal = 0 #0 for non-diagonal sudoku, 1 for diagonal sudoku
if isDiagonal == 1:
    unitlist = row_units + column_units + square_units + diagonal1_units + diagonal2_units
else:
    unitlist = row_units + column_units + square_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args: grid(string) - A grid in string form.
    Returns: A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value,
            then the value will be '123456789'.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def eliminate(values):
    """Eliminate values from peers of each box with a single value.
    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.
    Args:values: Sudoku in dictionary form.
    Returns:Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values=[box for box in values.keys() if len(values[box])==1]
    for box in solved_values:
        digit=values[box]
        for peer in peers[box]:
            values[peer]=values[peer].replace(digit,'')
    return values

def only_choice(values):
    '''Finalize all values that are the only choice for a unit.
    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.
    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    '''
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:the values dictionary with the naked twins eliminated from peers.
    """
    
    # Find all instances of naked twins
    all_two = set([box for box in values.keys() if len(values[box]) == 2])
    naked_twins = [[box1,box2] for box1 in all_two \
                    for box2 in peers[box1] \
                    if set(values[box1])==set(values[box2]) ]
    
    #print("len(naked_twins)=",len(naked_twins))
    if len(naked_twins)!=0:
        
        # Eliminate the naked twins as possibilities for their peers
        for i in range(len(naked_twins)):
            box1 = naked_twins[i][0]
            box2 = naked_twins[i][1]
            #print("box1",box1,"\nbox2",box2)
            if box1[0]==box2[0]:
                common_unit=box1[0]
                common_unit_peers= set(cross(common_unit, cols))-set(naked_twins[i])
            elif box1[1]==box2[1]:
                common_unit=box1[1]
                common_unit_peers= set(cross(rows, common_unit))-set(naked_twins[i])

            #print("common_unit_peers",common_unit_peers)

            # Delete the two digits in naked twins from all common peers.
            for peer_val in common_unit_peers:
                if len(values[peer_val])>2:
                    for rm_val in values[box1]:
                        values = assign_value(values, peer_val, values[peer_val].replace(rm_val,''))

    else : return values

    return values

def reduce_puzzle(values):
    '''Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    '''
    stalled=False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values


def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt
        
def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values



def solve(grid):
    """Find the solution to a Sudoku grid.
    Args: grid(string): a string representing a sudoku grid.
        Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns: The dictionary representation of the final sudoku grid.
            False if no solution exists.
    """
    values=grid_values(grid)
    
    print("Given sudoku:")
    display(values)
    
    '''
    values=eliminate(values)
    print("\nAfter eliminate:")
    display(values)
    print("\nAfter onlychoice:")
    values= only_choice(values)
    display(values)
    print("\nAfter nakedtwins:")
    values= naked_twins(values)
    display(values)
    '''
    '''
    print("\nAfter reduce:")
    values=reduce_puzzle(values) # eliminate, only_choice, naked_twins
    display(values)
    '''
    
    values=search(values)
    #print("\nAfter search:")
    #display(values)
    #print("\n")

    return values

if __name__ == '__main__':
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    reducegrid= '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
    searchneededgrid = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
    testgrid='9.1....8.8.5.7..4.2.4....6...7......5..............83.3..6......9................'

    
    #print("\nAfter solve:")
    display(solve(testgrid))


    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


Given sudoku:
    9     123456789     1     |123456789 123456789 123456789 |123456789     8     123456789 
    8     123456789     5     |123456789     7     123456789 |123456789     4     123456789 
    2     123456789     4     |123456789 123456789 123456789 |123456789     6     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     7     |123456789 123456789 123456789 |123456789 123456789 123456789 
    5     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |    8         3     123456789 
------------------------------+------------------------------+------------------------------
    3     123456789 123456789 |    6     123456789 123456789 |123456789 123456789 123456789 
123456789     9     123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 12345

In [221]:
#still one set of twins left. 
#Working pieces. Need to add Diagonal and nakedTwins
#refered to https://github.com/vxy10/AIND-Sudoku/blob/master/solution.py

rows = 'ABCDEFGHI'
cols = '123456789'

assignments = []

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s+t for s in A for t in B]

boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]


#Adding diagonal sudoku
diagonal1_units = [[rows[i]+cols[i] for i in range(len(rows))]]
diagonal2_units = [[rows[i]+cols[::-1][i] for i in range(len(rows))]]

isDiagonal = 1 #0 for non-diagonal sudoku, 1 for diagonal sudoku
if isDiagonal == 1:
    unitlist = row_units + column_units + square_units + diagonal1_units + diagonal2_units
else:
    unitlist = row_units + column_units + square_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args: grid(string) - A grid in string form.
    Returns: A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value,
            then the value will be '123456789'.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def eliminate(values):
    """Eliminate values from peers of each box with a single value.
    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.
    Args:values: Sudoku in dictionary form.
    Returns:Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values=[box for box in values.keys() if len(values[box])==1]
    for box in solved_values:
        digit=values[box]
        for peer in peers[box]:
            values[peer]=values[peer].replace(digit,'')
    return values

def only_choice(values):
    '''Finalize all values that are the only choice for a unit.
    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.
    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    '''
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:the values dictionary with the naked twins eliminated from peers.
    """
    
    # Find all instances of naked twins
    all_two = set([box for box in values.keys() if len(values[box]) == 2])
    naked_twins = [[box1,box2] for box1 in all_two \
                               for box2 in peers[box1] \
                    if set(values[box1])==set(values[box2]) ]
    
    #print("len(naked_twins)=",len(naked_twins))
    #print("naked_twins=",naked_twins)
    if len(naked_twins)!=0:
        #display(values)
        # Eliminate the naked twins as possibilities for their peers
        for i in range(len(naked_twins)):
            box1 = naked_twins[i][0]
            box2 = naked_twins[i][1]
            #print("box1",box1,values[box1],"\nbox2",box2,values[box2])
            if box1[0]==box2[0]:
                common_unit=box1[0]
                common_unit_peers= set(cross(common_unit, cols))-set(naked_twins[i])
            elif box1[1]==box2[1]:
                common_unit=box1[1]
                common_unit_peers= set(cross(rows, common_unit))-set(naked_twins[i])
            elif box1 in diagonal1_units[0] and box2 in diagonal1_units[0]: 
                common_unit_peers=set(diagonal1_units[0])-set(naked_twins[i])
            elif box1 in diagonal2_units[0] and box2 in diagonal2_units[0]: 
                common_unit_peers=set(diagonal2_units[0])-set(naked_twins[i])
            elif len([sunit for sunit in square_units if box1 in sunit and box2 in sunit])!=0:
                sunit=[sunit for sunit in square_units if box1 in sunit and box2 in sunit]
                common_unit_peers=set(sunit[0])-set(naked_twins[i])
                
            #print("common_unit_peers",sorted(common_unit_peers))

            # Delete the two digits in naked twins from all common peers.
            for peer_val in common_unit_peers:
                if len(values[peer_val])>2:
                    for rm_val in values[box1]:
                        values = assign_value(values, peer_val, values[peer_val].replace(rm_val,''))

    else : return values

    return values

def reduce_puzzle(values):
    '''Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    '''
    stalled=False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values


def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt
        
def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values


def solve(grid):
    """Find the solution to a Sudoku grid.
    Args: grid(string): a string representing a sudoku grid.
        Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns: The dictionary representation of the final sudoku grid.
            False if no solution exists.
    """
    values=grid_values(grid)
    
    print("Given sudoku:")
    display(values)
    
    '''
    values=eliminate(values)
    print("\nAfter eliminate:")
    display(values)
    print("\nAfter onlychoice:")
    values= only_choice(values)
    display(values)
    print("\nAfter nakedtwins:")
    values= naked_twins(values)
    display(values)
    '''
    '''
    print("\nAfter reduce:")
    values=reduce_puzzle(values) # eliminate, only_choice, naked_twins
    display(values)
    '''
    
    values=search(values)
    #print("\nAfter search:")
    #display(values)
    #print("\n")

    return values

if __name__ == '__main__':
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    reducegrid= '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
    searchneededgrid = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
    testgrid='9.1....8.8.5.7..4.2.4....6...7......5..............83.3..6......9................'
    testgrid2='5.....87.....5.4..9.....25....895....5....9..1.......5...5..............3..1.....'
    
    display(solve(testgrid2))


    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


Given sudoku:
    5     123456789 123456789 |123456789 123456789 123456789 |    8         7     123456789 
123456789 123456789 123456789 |123456789     5     123456789 |    4     123456789 123456789 
    9     123456789 123456789 |123456789 123456789 123456789 |    2         5     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789 123456789 |    8         9         5     |123456789 123456789 123456789 
123456789     5     123456789 |123456789 123456789 123456789 |    9     123456789 123456789 
    1     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     5     
------------------------------+------------------------------+------------------------------
123456789 123456789 123456789 |    5     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
    3     123456789 123456789 |    1     123456789 12345

In [207]:
print(diagonal1_units)
print(diagonal2_units)
print('B2' in diagonal1_units[0])
print(type(diagonal1_units[0]))
#print(set(diagonal1_units))
naked_twins=[['G2', 'I2'], ['I2', 'G2']]
print(naked_twins[0])
print("type(naked_twins[0])",type(naked_twins[0]))
print(set(naked_twins[0]))
print(set(diagonal2_units[0]))

[['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9']]
[['A9', 'B8', 'C7', 'D6', 'E5', 'F4', 'G3', 'H2', 'I1']]
True
<class 'list'>
['G2', 'I2']
type(naked_twins[0]) <class 'list'>
{'I2', 'G2'}
{'C7', 'E5', 'I1', 'F4', 'H2', 'D6', 'A9', 'G3', 'B8'}


In [174]:
print(sorted({'C1', 'I1', 'B1', 'H1', 'A1', 'E1', 'G1'}))

['A1', 'B1', 'C1', 'E1', 'G1', 'H1', 'I1']


In [195]:
pp.pprint(unitlist)

[ ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'],
  ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
  ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'],
  ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'],
  ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
  ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'],
  ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'],
  ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9'],
  ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
  ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
  ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
  ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
  ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
  ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'],
  ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'],
  ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'],
  ['A9', 'B9',

In [216]:
pp.pprint(square_units)
for sunit in square_units:
    print(sunit)

[ ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'],
  ['A4', 'A5', 'A6', 'B4', 'B5', 'B6', 'C4', 'C5', 'C6'],
  ['A7', 'A8', 'A9', 'B7', 'B8', 'B9', 'C7', 'C8', 'C9'],
  ['D1', 'D2', 'D3', 'E1', 'E2', 'E3', 'F1', 'F2', 'F3'],
  ['D4', 'D5', 'D6', 'E4', 'E5', 'E6', 'F4', 'F5', 'F6'],
  ['D7', 'D8', 'D9', 'E7', 'E8', 'E9', 'F7', 'F8', 'F9'],
  ['G1', 'G2', 'G3', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3'],
  ['G4', 'G5', 'G6', 'H4', 'H5', 'H6', 'I4', 'I5', 'I6'],
  ['G7', 'G8', 'G9', 'H7', 'H8', 'H9', 'I7', 'I8', 'I9']]
['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']
['A4', 'A5', 'A6', 'B4', 'B5', 'B6', 'C4', 'C5', 'C6']
['A7', 'A8', 'A9', 'B7', 'B8', 'B9', 'C7', 'C8', 'C9']
['D1', 'D2', 'D3', 'E1', 'E2', 'E3', 'F1', 'F2', 'F3']
['D4', 'D5', 'D6', 'E4', 'E5', 'E6', 'F4', 'F5', 'F6']
['D7', 'D8', 'D9', 'E7', 'E8', 'E9', 'F7', 'F8', 'F9']
['G1', 'G2', 'G3', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3']
['G4', 'G5', 'G6', 'H4', 'H5', 'H6', 'I4', 'I5', 'I6']
['G7', 'G8', 'G9', 'H7', 'H8', 'H9', '